# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




### Instalação e importação de bibliotecas

In [1]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate

In [2]:
# Para organizar resultados, gerar prints em amarelo

def print_y(text):
  """Imprime o texto fornecido na cor amarela.

  Args:
    text: O texto a ser impresso.
  """
  print(f"\033[33m ==> {text}\033[0m")

# Exemplo de uso:
print_y("Este texto será impresso em amarelo!")
print("Este em normal!")

 ==> Este texto será impresso em amarelo!
Este em normal!


# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [3]:
ocde_evit = pd.read_csv('files_in_geral/cid10_evit_ocde.csv')
ocde_evit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [4]:
dobr_evit_merged_temp = pd.read_parquet('files_in_geral/dobr_10_19a22_u75.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   |
|----+------------+-------------+--------+-----------+--------------+-------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       |
+----+------------+-------------+--------+-----------+--------------+-------------+


In [5]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_in_geral/dobr_10_19a22_u75.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


In [6]:
df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit
column_b = 'cid_pai'
column_return = 'avoid_flag'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(df_a)

       CAUSABAS        classificacao
0          A049  Avoidable mortality
1          A09   Avoidable mortality
2          A162  Avoidable mortality
3          A169  Avoidable mortality
4          A199  Avoidable mortality
...         ...                  ...
11890  W705      Avoidable mortality
11891  D047       causa não evitável
11892  C452      Avoidable mortality
11893  E065      Avoidable mortality
11894  P923      Avoidable mortality

[11895 rows x 2 columns]


In [7]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 4427475
Soma de óbitos : 4446922
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | classificacao       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Avoidable mortality |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Avoidable mortality |
+----+------------+-------------+--------+-----------+--------------+-------------+--------------

In [8]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('classificacao').count()

,CAUSABAS,CODMUNRES,SEXO,RACACOR,OBITO_SOMA,ANO_OBITO
classificacao,,,,,,
Avoidable mortality,2533757,2533757,2533757,2533757,2533757,2533757
causa não evitável,1893718,1893718,1893718,1893718,1893718,1893718


In [9]:
# Manter na base só dados de CID evitáveis
dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 2533757
Soma de óbitos : 2537888
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | classificacao       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Avoidable mortality |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Avoidable mortality |
+----+------------+-------------+--------+-----------+--------------+-------------+--------------

##########       Atenção      ##########

Atenção ao salvar o arquivo com título da versão e extensão adequada

In [10]:
# Salvar arquivo final de DOBR
dobr_evit_merged_temp02.to_parquet('files_temp/ds1_dobr_evit_merged.parquet')

### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [11]:
# Dados do Censo

# Verificar colunas de população no arquivo censo_pop_clean
censo_temp = pd.read_parquet('files_in_geral/censo_mun.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_temp
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111400
+----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------+
|    | Ano   | mun_co   | mun_no                     | Sexo   | Cor ou raça   |   pop_total |   pop_under74 | uf   | uf_co   | uf_no    |
|----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------|
|  0 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Branca        |        5311 |          5202 | RO   | _11     | Rondônia |
|  1 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Preta         |         693 |           673 | RO   | _11     | Rondônia |
|  2 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Amarela       |          96 |            94 | RO   | _11     | Rondônia |
|  3 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Parda         |        6323 |          6206 | RO   | _11     | Rondônia |
|  4 

In [12]:
# Dados de óbitos, arquivo com merge de cid evitáveis
dobr_merged_temp = pd.read_parquet('files_temp/ds1_dobr_evit_merged.parquet')


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 2533757
Soma de óbitos : 2537888
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | classificacao       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Avoidable mortality |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Avoidable mortality |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Avoidable mortality |
+----+------------+-------------+--------+-----------+--------------+-------------+--------------

##########         Etapa diferenciada da versão          ##########

Este bloco de código a seguir retira o CAUSABAS

In [13]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS
col_group= [
            # 'CAUSABAS', 
            'CODMUNRES', 'SEXO', 'RACACOR', 'ANO_OBITO']

dobr_merged_temp01 = dobr_merged_temp.groupby(col_group)['OBITO_SOMA'].sum().reset_index()

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 154730
Soma de óbitos : 2537888
+----+-------------+--------+-----------+-------------+--------------+
|    | CODMUNRES   |   SEXO | RACACOR   | ANO_OBITO   |   OBITO_SOMA |
|----+-------------+--------+-----------+-------------+--------------|
|  0 | _110000     |      1 |           | _2020       |            1 |
|  1 | _110000     |      1 |           | _2021       |            1 |
|  2 | _110000     |      1 | 1         | _2019       |            2 |
|  3 | _110000     |      1 | 1         | _2021       |            2 |
|  4 | _110000     |      1 | 1         | _2022       |            1 |
+----+-------------+--------+-----------+-------------+--------------+


In [14]:
# Tratamento de códigos de dobr para merge
print(f'Código de sexo: {dobr_merged_temp01['SEXO'].unique()}')
print(f'Código de racacor: {dobr_merged_temp01['RACACOR'].unique()}')

Código de sexo: ['1' '1   ' '2' '2   ' '0' '9   ']
Código de racacor: [' ' '1' '2' '4' '       ' '4      ' '1      ' '5' '2      ' '5      ' '3'
 '3      ']


In [15]:
# # Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp02 = dobr_merged_temp01.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# # # Criar um campo string que concatene ANO, CODMUNRES
# dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp['CODMUNRES']

# # mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 154730
+----+-------------+--------+-----------+-------------+--------------+-------------+
|    | CODMUNRES   |   SEXO | RACACOR   | ANO_OBITO   |   OBITO_SOMA | ANO_PROXY   |
|----+-------------+--------+-----------+-------------+--------------+-------------|
|  0 | _110000     |      1 |           | _2020       |            1 | _2022       |
|  1 | _110000     |      1 |           | _2021       |            1 | _2022       |
|  2 | _110000     |      1 | 1         | _2019       |            2 | _2022       |
|  3 | _110000     |      1 | 1         | _2021       |            2 | _2022       |
|  4 | _110000     |      1 | 1         | _2022       |            1 | _2022       |
+----+-------------+--------+-----------+-------------+--------------+-------------+


In [16]:
# Transformar valores RACACOR e SEXO
dobr_merged_temp03 = dobr_merged_temp01.copy()

# Transformar valores RACACOR
# dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].replace([' ', ''], '0')
# dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].fillna('0')
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].astype(str)
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].str[:1]
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].replace(' ','9')
dobr_merged_temp03['RACACOR'] = '_' + dobr_merged_temp03['RACACOR'].astype(str)

# Transformar valores SEXO
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].astype(str)
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].str[:1]
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].replace('0','9')
dobr_merged_temp03['SEXO'] = '_' + dobr_merged_temp03['SEXO'].astype(str)

dobr_merged_temp03['ANO_PROXY'] = dobr_merged_temp03['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')


print(f'Código de RACACOR: {dobr_merged_temp03["RACACOR"].unique()}')
print(f'Código de SEXO: {dobr_merged_temp03["SEXO"].unique()}')
print(f'Código de ANO: {dobr_merged_temp03['ANO_OBITO'].unique()}')
print(f'Código de ANO_PROXY: {dobr_merged_temp03['ANO_PROXY'].unique()}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')

Código de RACACOR: ['_9' '_1' '_2' '_4' '_5' '_3']
Código de SEXO: ['_1' '_2' '_9']
Código de ANO: ['_2020' '_2021' '_2019' '_2022' '_2010']
Código de ANO_PROXY: ['_2022' '_2010']
Total de óbitos : 2537888


In [17]:
# Apresentar total de registros de DOBR sem identificação de SEXO e RACACOR
dobr_merged_temp03_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['SEXO'] == '_9') | (dobr_merged_temp03['RACACOR'] == '_9')]
sexo_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['SEXO']== '_9')]
racacor_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['RACACOR']== '_9')]

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03_fault
print(f'Registros de RACACOR sem qualidade : {racacor_fault.OBITO_SOMA.sum()}')
print(f'Registros de SEXO sem qualidade : {sexo_fault.OBITO_SOMA.sum()}')
print(f'Registros total sem qualidade : {tab.OBITO_SOMA.sum()}')

faults_perc = (tab.OBITO_SOMA.sum()/dobr_merged_temp03.OBITO_SOMA.sum())*100

print_y(f'Total de óbitos sem dados de qualidade em SEXO e RACACOR: {tab.OBITO_SOMA.sum()}')
print_y(f'Percentual de óbitos sem dados de qualidade em SEXO e RACACOR: {faults_perc}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros de RACACOR sem qualidade : 76293
Registros de SEXO sem qualidade : 573
Registros total sem qualidade : 76703
 ==> Total de óbitos sem dados de qualidade em SEXO e RACACOR: 76703
 ==> Percentual de óbitos sem dados de qualidade em SEXO e RACACOR: 3.0223161936224137


In [18]:
# Excluir valores sem qualidade da tabela
dobr_merged_temp04 = dobr_merged_temp03[(dobr_merged_temp03['RACACOR'] != '_9') & (dobr_merged_temp03['SEXO'] != '_9')]

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')

print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros da tabela : 135976
Total de óbitos : 2461185
+----+-------------+--------+-----------+-------------+--------------+-------------+
|    | CODMUNRES   | SEXO   | RACACOR   | ANO_OBITO   |   OBITO_SOMA | ANO_PROXY   |
|----+-------------+--------+-----------+-------------+--------------+-------------|
|  2 | _110000     | _1     | _1        | _2019       |            2 | _2022       |
|  3 | _110000     | _1     | _1        | _2021       |            2 | _2022       |
|  4 | _110000     | _1     | _1        | _2022       |            1 | _2022       |
|  5 | _110000     | _1     | _2        | _2019       |            1 | _2022       |
|  6 | _110000     | _1     | _2        | _2022       |            1 | _2022       |
+----+-------------+--------+-----------+-------------+--------------+-------------+


In [19]:
# Transformar campos de cor e sexo em código
censo_temp01 = censo_temp.copy()

# Transformar valores de sexo em código (utilizado pelo SIM)
censo_temp01['SEXO'] = censo_temp01['Sexo'].map({'Homens':'_1',
                                                    'Mulheres':'_2'})
# Transformar valores nan em 0
censo_temp01['RACACOR'] = censo_temp01['Cor ou raça'].fillna('_0')

# Transformar valores de racacor em código (utilizado pelo SIM)
censo_temp01['RACACOR'] = censo_temp01['RACACOR'].map({'Branca':'_1',
                                                        'Preta':'_2',
                                                        'Amarela':'_3',
                                                        'Parda':'_4',
                                                        'Indígena':'_5',
                                                        'Sem declaração':'_9'})

# Trasformar valores de ano
censo_temp01['ANO_OBITO_'] = censo_temp01['Ano'].astype(str)

# Exibir resultados
print(f'Código de sexo: {censo_temp01['SEXO'].unique()}')
print(f'Código de racacor: {censo_temp01['RACACOR'].unique()}')
print(f'Código de ANO: {censo_temp01['ANO_OBITO_'].unique()}')

Código de sexo: ['_1' '_2']
Código de racacor: ['_1' '_2' '_3' '_4' '_5']
Código de ANO: ['_2010' '_2022']


In [20]:
# Realizar o merge dos dataframes
dobr_merged_temp05 = pd.merge(dobr_merged_temp04, censo_temp01, 
                              left_on=['ANO_PROXY','CODMUNRES','SEXO','RACACOR'],
                              right_on=['ANO_OBITO_','mun_co','SEXO','RACACOR'],
                              how='left')


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp05
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 135976
Total de óbitos : 2461185


,CODMUNRES,SEXO,RACACOR,ANO_OBITO,OBITO_SOMA,ANO_PROXY,Ano,mun_co,mun_no,Sexo,Cor ou raça,pop_total,pop_under74,uf,uf_co,uf_no,ANO_OBITO_
0,_110000,_1,_1,_2019,2,_2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_110000,_1,_1,_2021,2,_2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,_110000,_1,_1,_2022,1,_2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,_110000,_1,_2,_2019,1,_2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,_110000,_1,_2,_2022,1,_2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp06 = dobr_merged_temp05.dropna(subset=['pop_under74']).reset_index(drop=True)

dobr_merged_temp06 = dobr_merged_temp06[['ANO_OBITO','uf_no','mun_no','Sexo','Cor ou raça','OBITO_SOMA','pop_total','pop_under74']]


tab = dobr_merged_temp06
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))


Registros da tabela : 135626
Total de óbitos : 2457520
+----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------+
|    | ANO_OBITO   | uf_no    | mun_no                     | Sexo   | Cor ou raça   |   OBITO_SOMA |   pop_total |   pop_under74 |
|----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------|
|  0 | _2019       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           17 |        3770 |          3606 |
|  1 | _2020       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           24 |        3770 |          3606 |
|  2 | _2021       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |            9 |        3770 |          3606 |
|  3 | _2022       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           23 |        3770 |          3606 |
|  4 | _2019       | Rondôni

In [22]:
# Exibir resultados
print(f'Código de sexo: {dobr_merged_temp06['Sexo'].unique()}')
print(f'Código de racacor: {dobr_merged_temp06['Cor ou raça'].unique()}')
# print(f'Código de ANO: {dobr_merged_temp06['ANO_PROXY'].unique()}')
print(f'Código de ANO: {dobr_merged_temp06['ANO_OBITO'].unique()}')

Código de sexo: ['Homens' 'Mulheres']
Código de racacor: ['Branca' 'Preta' 'Parda' 'Indígena' 'Amarela']
Código de ANO: ['_2019' '_2020' '_2021' '_2022' '_2010']


##### Arquivo tratado final

In [23]:
# Contagem de valores com código de município sem qualidade
mun_no_nulo = dobr_merged_temp05[dobr_merged_temp05.mun_no.isna()]

tab = mun_no_nulo
# print(f'Registros da tabela excluídos por qualidade no código de município: {tab.shape[0]}')
print_y(f'Total de óbitos excluídos por qualidade no código de município: {tab.OBITO_SOMA.sum()}')

 ==> Total de óbitos excluídos por qualidade no código de município: 3665


##########         Atenção        ##########

Atenção da hora de salvar o arquivo, com as informações corretas de versão

In [24]:
# Salvar arquivo
dobr_merged_temp06.to_parquet('files_temp/ds1_dobr_merged.parquet')

In [25]:
# Filtrar registros onde pop_under74 é null
dobr_merged_pop_nulos = dobr_merged_temp06[dobr_merged_temp06['pop_under74'].isnull()]

# Exibir os registros filtrados
dobr_merged_pop_nulos['OBITO_SOMA'].sum()

np.int64(0)

# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 1 - UF/Mun/sexo_raça


O primeiro dataset que iremos construir tem as seguintes características:

* setting: UF
* dimension: município
* subgroup: sexo + raça-cor

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [ ]:
# Mostrar colunas do template do HEAT

import requests
import openpyxl

# Baixando o arquivo
url = 'https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm'
response = requests.get(url)

# Salvando o arquivo localmente (você pode escolher outro nome)
with open('arquivo_excel.xlsm', 'wb') as f:
    f.write(response.content)

# Abrindo o arquivo Excel
workbook = openpyxl.load_workbook('arquivo_excel.xlsm')

# Selecionando a primeira aba
sheet = workbook.template
sheet = workbook['template']

# Mostrar conteúdo da aba 'template'
for row in sheet.iter_rows(values_only=True):
    print(row)



('setting', 'date', 'source', 'indicator_abbr', 'indicator_name', 'dimension', 'subgroup', 'estimate', 'se', 'ci_lb', 'ci_ub', 'population', 'note', 'setting_average', 'iso3', 'favourable_indicator', 'indicator_scale', 'ordered_dimension', 'subgroup_order', 'reference_subgroup')


In [26]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_temp/ds1_dobr_merged.parquet')

# mostrar resultados
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 135626
Total de óbitos : 2457520
+----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------+
|    | ANO_OBITO   | uf_no    | mun_no                     | Sexo   | Cor ou raça   |   OBITO_SOMA |   pop_total |   pop_under74 |
|----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------|
|  0 | _2019       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           17 |        3770 |          3606 |
|  1 | _2020       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           24 |        3770 |          3606 |
|  2 | _2021       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |            9 |        3770 |          3606 |
|  3 | _2022       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           23 |        3770 |          3606 |
|  4 | _2019       | Rondôni

In [27]:
heatds_temp.columns

Index(['ANO_OBITO', 'uf_no', 'mun_no', 'Sexo', 'Cor ou raça', 'OBITO_SOMA',
       'pop_total', 'pop_under74'],
      dtype='object')

##########       Atenção      ##########

Criação do indicador de mortalidade por ano, município, sexo e raça

In [28]:
heatds_temp01 = heatds_temp.copy()

# Gerar o indicador de mortalidade
heatds_temp01['mort_ind_mun'] = (heatds_temp01['OBITO_SOMA']/heatds_temp01['pop_under74'])*100000

# mostrar resultados
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.info()

Registros da tabela : 135626
Total de óbitos : 2457520
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135626 entries, 0 to 135625
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ANO_OBITO     135626 non-null  object 
 1   uf_no         135626 non-null  object 
 2   mun_no        135626 non-null  object 
 3   Sexo          135626 non-null  object 
 4   Cor ou raça   135626 non-null  object 
 5   OBITO_SOMA    135626 non-null  int64  
 6   pop_total     135626 non-null  float64
 7   pop_under74   135626 non-null  float64
 8   mort_ind_mun  135626 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 9.3+ MB


In [29]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun']).sum()

# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
registros_inf

# mostrar resultados
tab = registros_inf
print(f'Registros da tabela excluidos por valores infinitos : {tab.shape[0]}')
print(f'Total de óbitos excluídos por valores infinitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Número de valores infinitos em mort_ind_mun: 167
Registros da tabela excluidos por valores infinitos : 167
Total de óbitos excluídos por valores infinitos : 175


In [30]:
# Excluir registros com valores infinitos em mort_ind_mun
heatds_temp02 = heatds_temp01[~np.isinf(heatds_temp01['mort_ind_mun'])].reset_index(drop=True)

# mostrar resultados
tab = heatds_temp02
print(f'Registros da tabela após excluir valores infinitos : {tab.shape[0]}')
print(f'Total de óbitos após excluir valores infinitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela após excluir valores infinitos : 135459
Total de óbitos após excluir valores infinitos : 2457345


In [31]:
heatds_temp02.describe()

,OBITO_SOMA,pop_total,pop_under74,mort_ind_mun
count,135459.000000,1.354590e+05,1.354590e+05,135459.000000
mean,18.140877,7.252330e+03,6.969796e+03,689.415598
std,113.248620,4.337311e+04,4.133199e+04,4547.196370
min,1.000000,1.000000e+00,1.000000e+00,8.345156
25%,2.000000,8.370000e+02,7.940000e+02,155.038760
50%,4.000000,1.948000e+03,1.863000e+03,256.959315
75%,11.000000,4.872500e+03,4.677000e+03,418.410042
max,11705.000000,3.651217e+06,3.460633e+06,300000.000000


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [32]:
# Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
dobr_grouped_uf = heatds_temp02.groupby(['uf_no',
                                            #  'uf_co',
                                             'ANO_OBITO',
                                            #  'SEXO',
                                            #  'RACACOR'
                                             ]).agg({'OBITO_SOMA': 'sum',
                                                    'pop_under74': 'sum',
                                                    # 'mort_ind_mun': 'mean'
                                                }).reset_index()
dobr_grouped_uf.rename(columns={
                                # 'mort_ind_mun': 'mort_ind_uf',
                                'OBITO_SOMA': 'OBITO_SOMA_uf',
                                'pop_under74': 'pop_under74_uf',
                                }, inplace=True)

# Gerar o indicador de mortalidade
dobr_grouped_uf['mort_ind_uf'] = (dobr_grouped_uf['OBITO_SOMA_uf']/dobr_grouped_uf['pop_under74_uf'])*100000

# mostrar resultados
tab = dobr_grouped_uf
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA_uf.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 135
Total de óbitos : 2457345
+----+---------+-------------+-----------------+------------------+---------------+
|    | uf_no   | ANO_OBITO   |   OBITO_SOMA_uf |   pop_under74_uf |   mort_ind_uf |
|----+---------+-------------+-----------------+------------------+---------------|
|  0 | Acre    | _2010       |            1101 |           683087 |       161.18  |
|  1 | Acre    | _2019       |            1865 |           782221 |       238.424 |
|  2 | Acre    | _2020       |            1705 |           777464 |       219.303 |
|  3 | Acre    | _2021       |            1849 |           783842 |       235.889 |
|  4 | Acre    | _2022       |            1689 |           781302 |       216.178 |
+----+---------+-------------+-----------------+------------------+---------------+


In [33]:
dobr_grouped_uf.describe()

,OBITO_SOMA_uf,pop_under74_uf,mort_ind_uf
count,135.000000,1.350000e+02,135.000000
mean,18202.555556,6.993493e+06,253.948892
std,21352.781771,8.224015e+06,32.446394
min,870.000000,4.268220e+05,161.180055
25%,5388.000000,2.615825e+06,238.426410
50%,9106.000000,3.640631e+06,255.382807
75%,24271.000000,8.498988e+06,270.665252
max,112248.000000,4.208267e+07,328.841521


##########       Atenção      ##########

Salvar arquivo com indicador de mortalidade por ano, uf, sexo, raça

Atenção ao salvar arquivo

In [34]:
# Salvar arquivo por UF para exportação
dobr_grouped_uf.to_parquet('files_temp/ds1_heatds_temp_uf.parquet')

##########       Atenção      ##########

Atenção ao merge de valores quer serão setting-average

In [35]:
# Trazer dados por UF (setting e setting_average)

# Realizar o merge entre heat_ e uf_sett
heatds_temp03 = pd.merge(heatds_temp02, dobr_grouped_uf, on=['uf_no',
                                                                'ANO_OBITO',
                                                                # 'SEXO',
                                                                # 'RACACOR'
                                                                ], how='left')
# heatds_temp03.rename(columns={'mort_ind': 'mort_ind_uf'}, inplace=True)
# Exibir as primeiras linhas do dataframe resultante

# mostrar resultados
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 135459
Total de óbitos : 2457345
+----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------+----------------+-----------------+------------------+---------------+
|    | ANO_OBITO   | uf_no    | mun_no                     | Sexo   | Cor ou raça   |   OBITO_SOMA |   pop_total |   pop_under74 |   mort_ind_mun |   OBITO_SOMA_uf |   pop_under74_uf |   mort_ind_uf |
|----+-------------+----------+----------------------------+--------+---------------+--------------+-------------+---------------+----------------+-----------------+------------------+---------------|
|  0 | _2019       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           17 |        3770 |          3606 |        471.436 |            3602 |      1.50549e+06 |       239.257 |
|  1 | _2020       | Rondônia | Alta Floresta D'Oeste (RO) | Homens | Branca        |           24 |        3770 |          3606 |        665

In [36]:
# Teste de aplicação de setting_average (médias por UF)
filtered_data = heatds_temp03[   (heatds_temp03['ANO_OBITO'] == '_2022') &
                                (heatds_temp03['uf_no'] == 'Acre') 
                                # & (heat_treat00['SEXO'] == '_1') &
                                #(heat_treat00['RACACOR'] == '_4')
                              ]
filtered_data.head()

,ANO_OBITO,uf_no,mun_no,Sexo,Cor ou raça,OBITO_SOMA,pop_total,pop_under74,mort_ind_mun,OBITO_SOMA_uf,pop_under74_uf,mort_ind_uf
1440,_2022,Acre,Acrelândia (AC),Homens,Branca,1,1797.0,1733.0,57.703405,1689,781302.0,216.177611
1447,_2022,Acre,Acrelândia (AC),Homens,Parda,23,4362.0,4262.0,539.652745,1689,781302.0,216.177611
1454,_2022,Acre,Acrelândia (AC),Mulheres,Branca,5,1680.0,1624.0,307.881773,1689,781302.0,216.177611
1460,_2022,Acre,Acrelândia (AC),Mulheres,Parda,4,4426.0,4366.0,91.617041,1689,781302.0,216.177611
1464,_2022,Acre,Assis Brasil (AC),Homens,Branca,1,635.0,619.0,161.550889,1689,781302.0,216.177611


In [37]:
heatds_temp03.columns

Index(['ANO_OBITO', 'uf_no', 'mun_no', 'Sexo', 'Cor ou raça', 'OBITO_SOMA',
       'pop_total', 'pop_under74', 'mort_ind_mun', 'OBITO_SOMA_uf',
       'pop_under74_uf', 'mort_ind_uf'],
      dtype='object')

##### Criar colunas do dataset

In [38]:
# Gerar colunas sem informação ou com informação padronizada
# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp04 = heatds_temp03.copy() # caso seja agrupado por UF anteriormente


heatds_temp04['indicator_abbr'] = 'AVM'
heatds_temp04['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp04['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp04['favourable_indicator'] = 0
heatds_temp04['indicator_scale'] = 100000
heatds_temp04['ordered_dimension'] = 0
heatds_temp04['subgroup_order'] = 0
heatds_temp04['reference_subgroup'] = 0
heatds_temp04['note'] = ''
heatds_temp04['se'] = ''
heatds_temp04['ci_lb'] = ''
heatds_temp04['ci_ub'] = ''
heatds_temp04['iso3'] = ''

heatds_temp04['subgroup'] = heatds_temp04['Sexo'] + ' - raça/cor '  +  heatds_temp04['Cor ou raça']



# mostrar resultados
tab = heatds_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.info()


Registros da tabela : 135459
Total de óbitos : 2457345
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135459 entries, 0 to 135458
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ANO_OBITO             135459 non-null  object 
 1   uf_no                 135459 non-null  object 
 2   mun_no                135459 non-null  object 
 3   Sexo                  135459 non-null  object 
 4   Cor ou raça           135459 non-null  object 
 5   OBITO_SOMA            135459 non-null  int64  
 6   pop_total             135459 non-null  float64
 7   pop_under74           135459 non-null  float64
 8   mort_ind_mun          135459 non-null  float64
 9   OBITO_SOMA_uf         135459 non-null  int64  
 10  pop_under74_uf        135459 non-null  float64
 11  mort_ind_uf           135459 non-null  float64
 12  indicator_abbr        135459 non-null  object 
 13  indicator_name        135459 non-null  object 
 1

In [39]:
# Verificar subgroup
heatds_temp04['subgroup'].unique()

array(['Homens - raça/cor Branca', 'Homens - raça/cor Preta',
       'Homens - raça/cor Parda', 'Homens - raça/cor Indígena',
       'Mulheres - raça/cor Branca', 'Mulheres - raça/cor Preta',
       'Mulheres - raça/cor Parda', 'Mulheres - raça/cor Indígena',
       'Homens - raça/cor Amarela', 'Mulheres - raça/cor Amarela'],
      dtype=object)

In [40]:
heatds_temp04.columns

Index(['ANO_OBITO', 'uf_no', 'mun_no', 'Sexo', 'Cor ou raça', 'OBITO_SOMA',
       'pop_total', 'pop_under74', 'mort_ind_mun', 'OBITO_SOMA_uf',
       'pop_under74_uf', 'mort_ind_uf', 'indicator_abbr', 'indicator_name',
       'source', 'favourable_indicator', 'indicator_scale',
       'ordered_dimension', 'subgroup_order', 'reference_subgroup', 'note',
       'se', 'ci_lb', 'ci_ub', 'iso3', 'subgroup'],
      dtype='object')

In [41]:
# Renomear colunas de heat_treat01
heatds_temp05 = heatds_temp04.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'uf_no':'setting',
    'mun_no':'dimension',
    'mort_ind_mun':'estimate',
    'mort_ind_uf':'setting_average'
})

# ajustes de colunas
heatds_temp05.date = heatds_temp05.date.str[1:5]
heatds_temp05['estimate'] = heatds_temp05['estimate'].astype(int)
heatds_temp05['setting_average'] = heatds_temp05['setting_average'].astype(int)

heatds_temp05.head()

,date,setting,dimension,Sexo,Cor ou raça,OBITO_SOMA,pop_total,population,estimate,OBITO_SOMA_uf,...,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3,subgroup
0,2019,Rondônia,Alta Floresta D'Oeste (RO),Homens,Branca,17,3770.0,3606.0,471,3602,...,100000,0,0,0,,,,,,Homens - raça/cor Branca
1,2020,Rondônia,Alta Floresta D'Oeste (RO),Homens,Branca,24,3770.0,3606.0,665,3777,...,100000,0,0,0,,,,,,Homens - raça/cor Branca
2,2021,Rondônia,Alta Floresta D'Oeste (RO),Homens,Branca,9,3770.0,3606.0,249,3938,...,100000,0,0,0,,,,,,Homens - raça/cor Branca
3,2022,Rondônia,Alta Floresta D'Oeste (RO),Homens,Branca,23,3770.0,3606.0,637,4198,...,100000,0,0,0,,,,,,Homens - raça/cor Branca
4,2019,Rondônia,Alta Floresta D'Oeste (RO),Homens,Preta,2,743.0,700.0,285,3602,...,100000,0,0,0,,,,,,Homens - raça/cor Preta


In [42]:
heatds_temp05.estimate.describe()

count    135459.000000
mean        688.919178
std        4547.223694
min           8.000000
25%         155.000000
50%         256.000000
75%         418.000000
max      300000.000000
Name: estimate, dtype: float64

In [43]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  

# Reorder the columns in the DataFrame
heatds_temp06 = heatds_temp05[column_order]


# mostrar dados gerais da tabela após transformações
tab = heatds_temp06
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 135459


In [227]:
# # Selecionar registros onde a coluna 'date' é igual a 2020, 2021 ou 2022
# heatds_temp06_x = heatds_temp06[heatds_temp06['date'].isin(['2022'])].reset_index(drop=True)

# # Mostrar resultados
# print(f'Registros da tabela : {heatds_temp06_x.shape[0]}')
# # print(tabulate(heatds_temp06_20a22.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 557860


##########       Atenção      ##########

Salvar arquivo para todos os anos (2010,2019,2020,2021,2022) como csv, para teste de carga

In [1]:
# ### Arquivo pronto para exportação
# heatds_temp06.to_csv('files_out/ds1_UF_mun_sexo&cor_10_19a22.csv', index=False)

NameError: name 'heatds_temp06' is not defined

In [44]:
heatds_temp06.to_parquet('files_out/ds1_UF_mun_sexo&cor_10_19a22.parquet')

In [ ]:
### Arquivo pronto para exportação
heatds_temp06.to_excel('files_out/ds1_UF_mun_sexo&cor_10_19a22.xlsx', index=False)

In [47]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('files_in_geral/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('files_out/ds1_UF_mun_sexo&cor_10_19a22.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('files_out/heat_plus_template_ds_4val.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados